<a href="https://colab.research.google.com/github/hongjinkong/opensw/blob/main/1204-1/deepfakedetection_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/mystlee/dfdc_deepfake_challenge.git

Cloning into 'dfdc_deepfake_challenge'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 134 (delta 23), reused 4 (delta 4), pack-reused 65 (from 1)
Receiving objects: 100% (134/134), 69.20 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Updating files: 100% (63/63), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Kaggle API 설정
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json  # 권한 설정

!kaggle datasets download -d manjilkarki/deepfake-and-real-images --force

import zipfile

# 압축 해제
with zipfile.ZipFile('deepfake-and-real-images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/deepfake_and_real_images')

# 데이터 확인
!ls /content/deepfake_and_real_images

!cp -r /content/deepfake_and_real_images /content/drive/MyDrive/

dataset_dir = '/content/drive/MyDrive/deepfake_and_real_images/Dataset'

# 데이터 확인
import os
print(os.listdir(dataset_dir))


Mounted at /content/drive
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
100% 1.68G/1.68G [00:16<00:00, 129MB/s]
100% 1.68G/1.68G [00:16<00:00, 108MB/s]
Dataset
cp: cannot stat '/content/drive/MyDrive/deepfake_and_real_images/Dataset/Train/Fake/fake_0.jpg': Input/output error
['Test', 'Train', 'Validation']


In [ ]:
import os
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Kaggle API 파일 설정
os.makedirs('/root/.kaggle', exist_ok=True)

# Kaggle API 키 업로드: kaggle.json 파일을 Colab에 업로드한 후 실행
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/  # Google Drive의 위치로 변경
!chmod 600 /root/.kaggle/kaggle.json  # 권한 설정

# Kaggle 데이터셋 다운로드
!kaggle datasets download -d manjilkarki/deepfake-and-real-images --force

# 압축 해제
!unzip -o deepfake-and-real-images.zip -d /content/dataset

# 데이터셋 위치 확인
dataset_dir = '/content/dataset/Dataset'
print(f"Dataset extracted to: {dataset_dir}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 경로를 정확히 설정
dataset_dir = '/content/deepfake_and_real_images/Dataset'  # 실제 경로로 변경

# 이미지 데이터 생성기 (ImageDataGenerator)
datagen = ImageDataGenerator(
    rescale=1./255,          # 이미지 픽셀 값을 0~1로 정규화
    rotation_range=15,       # 이미지 회전
    width_shift_range=0.1,   # 가로 이동
    height_shift_range=0.1,  # 세로 이동
    horizontal_flip=True     # 수평 뒤집기
)

# train, validation, test 데이터 로드 (디렉토리 구조에 맞춰 자동으로 라벨링)
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),  # 'Train' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류 (Fake: 0, Real: 1)
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),  # 'Validation' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),  # 'Test' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator 설정
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# 데이터 로드
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# 모델 정의 (이미지 분류 모델)
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

def build_classifier_with_dropout(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(flat)  # L2 정규화 추가
    dropout = layers.Dropout(0.5)(dense)  # 드롭아웃 추가
    output = layers.Dense(2, activation='softmax')(dropout)  # 두 클래스에 대한 softmax 활성화 함수
    classifier = Model(encoder.input, output, name="classifier_with_dropout")
    return classifier

# 모델 학습
input_shape = (128, 128, 3)
classifier = build_classifier_with_dropout(input_shape)
classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
classifier.summary()

# 모델 학습
epochs = 50  # 에포크 수
classifier.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,  # validation 데이터로 모델 평가
)

# Grad-CAM heatmap을 계산하는 함수
def get_gradcam_heatmap(model, image, class_idx):
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    if not last_conv_layer_name:
        raise ValueError("No Conv2D layer found in the model.")

    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    image_input = np.expand_dims(image, axis=0)
    image_input = tf.convert_to_tensor(image_input, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(image_input)
        conv_output = last_conv_layer_model(image_input)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Grad-CAM + Bounding Box 시각화 함수
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)

    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Grad-CAM 시각화
test_image, label = next(test_generator)
predicted_class = np.argmax(classifier.predict(test_image), axis=-1)[0]  # 다중 클래스 분류에서 예측된 클래스 선택
display_gradcam_with_bbox(classifier, test_image[0], predicted_class, label, threshold=0.5)
print(f"Predicted Class: {predicted_class}, Label: {label}")


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


Model: "classifier_with_dropout"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       2,097,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,190,594 (8.36 MB)

 Trainable params: 2,190,594 (8.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4375/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5982 - loss: 0.6607

KeyboardInterrupt: 

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# 데이터셋 경로 설정
dataset_dir = '/content/deepfake_and_real_images/Dataset'

# ImageDataGenerator 설정
datagen = ImageDataGenerator(
    rescale=1./255,         # 이미지 정규화
    rotation_range=15,      # 이미지 회전
    width_shift_range=0.1,  # 가로 이동
    height_shift_range=0.1, # 세로 이동
    horizontal_flip=True    # 수평 뒤집기
)

# 데이터 로드 및 설정
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),
    target_size=(96, 96),   # 이미지 크기
    batch_size=16,          # 배치 크기
    class_mode='categorical',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),
    target_size=(96, 96),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),
    target_size=(96, 96),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

# 훈련 샘플 수와 steps_per_epoch 설정
train_samples = 800
steps_per_epoch = train_samples // train_generator.batch_size

# 모델 정의
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

def build_classifier_with_dropout(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(flat)
    dropout = layers.Dropout(0.5)(dense)
    output = layers.Dense(2, activation='softmax')(dropout)  # 두 클래스 (Fake, Real)
    return Model(encoder.input, output, name="classifier_with_dropout")

# 모델 생성
input_shape = (96, 96, 3)
model = build_classifier_with_dropout(input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
model.summary()

# 모델 학습
epochs = 5  # 테스트용으로 5에포크로 설정
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch
)

# 모델 평가
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Grad-CAM heatmap 계산 함수
def get_gradcam_heatmap(model, image, class_idx):
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    if not last_conv_layer_name:
        raise ValueError("No Conv2D layer found in the model.")

    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    image_input = np.expand_dims(image, axis=0)
    image_input = tf.convert_to_tensor(image_input, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(image_input)
        conv_output = last_conv_layer_model(image_input)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Grad-CAM + Bounding Box 시각화 함수
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)

    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Grad-CAM 시각화
test_image, label = next(test_generator)
predicted_class = np.argmax(model.predict(test_image), axis=-1)[0]
display_gradcam_with_bbox(model, test_image[0], predicted_class, label, threshold=0.5)
print(f"Predicted Class: {predicted_class}, Label: {label}")


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import zipfile
import matplotlib.pyplot as plt

# 1. Google Drive 마운트 및 데이터셋 준비
def setup_dataset_from_drive(drive_path, local_extract_path):
    # Google Drive 마운트
    drive.mount('/content/drive')
    zip_path = os.path.join('/content/drive/My Drive', drive_path)

    # 압축 해제
    if not os.path.exists(local_extract_path):
        os.makedirs(local_extract_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(local_extract_path)

    print(f"Dataset extracted to {local_extract_path}")
    return local_extract_path

# 2. 프레임 추출 및 라벨링
def extract_frames_with_labels(video_dir, output_base_dir, frame_rate=5):
    # "fake"와 "real" 폴더에서 동영상 검색
    for label in ['fake', 'real']:
        label_dir = os.path.join(video_dir, label)
        output_label_dir = os.path.join(output_base_dir, label)
        os.makedirs(output_label_dir, exist_ok=True)

        for video_file in os.listdir(label_dir):
            video_path = os.path.join(label_dir, video_file)
            cap = cv2.VideoCapture(video_path)
            count = 0
            frame_id = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                if count % frame_rate == 0:  # frame_rate마다 프레임 추출
                    frame_path = os.path.join(output_label_dir, f"{video_file}_frame_{frame_id}.jpg")
                    cv2.imwrite(frame_path, frame)
                    frame_id += 1
                count += 1
            cap.release()
    print(f"Frames extracted to {output_base_dir}")

# 3. 데이터 로드
def load_data_from_frames(base_dir):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        validation_split=0.2  # Train/Validation Split
    )
    train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, val_generator

# 4. 모델 정의
def build_classifier(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# 5. 새로운 영상 판별
def predict_video(model, video_path, frame_rate=5):
    frames = extract_frames(video_path, output_dir="temp_frames", frame_rate=frame_rate)
    frames = frames / 255.0  # Normalize
    predictions = model.predict(frames)
    fake_probabilities = predictions[:, 1]  # Fake 클래스 확률
    avg_fake_probability = np.mean(fake_probabilities)

    if avg_fake_probability > 0.5:
        result = "Deepfake"
    else:
        result = "Real"

    print(f"Video Prediction: {result} (Average Fake Probability: {avg_fake_probability:.2f})")
    return result, avg_fake_probability

# 6. 실행
if __name__ == '__main__':
    # Google Drive에서 데이터셋 준비
    drive_path = 'dataset/dfdc_train_part_00.zip'  # Google Drive 내의 ZIP 파일 경로
    extract_path = './train_data'  # 로컬로 압축 해제할 디렉토리
    dataset_path = setup_dataset_from_drive(drive_path, extract_path)

    # 프레임 추출
    output_frames_dir = './train_frames'
    extract_frames_with_labels(dataset_path, output_frames_dir, frame_rate=5)

    # 데이터 로드
    train_gen, val_gen = load_data_from_frames(output_frames_dir)

    # 모델 생성 및 컴파일
    input_shape = (128, 128, 3)
    model = build_classifier(input_shape)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 훈련
    model.fit(train_gen, epochs=10, validation_data=val_gen)

    # 새로운 동영상 판별
    test_video_path = './test_video.mp4'  # 테스트할 동영상
    predict_video(model, test_video_path, frame_rate=10)
